# 🖼️ Introduction aux Réseaux Convolutifs (CNN)

Bienvenue dans le monde des **CNN (Convolutional Neural Networks)** !

## 🎯 Objectifs

1. **Comprendre pourquoi** les CNN sont meilleurs pour les images
2. **Apprendre les couches** : Convolution, Pooling
3. **Visualiser** ce que font ces opérations
4. **Implémenter** un CNN simple from scratch
5. **Comparer** MLP vs CNN sur MNIST
6. **Atteindre 98-99%** d'accuracy ! 🏆

---

In [ ]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import sys
from scipy.signal import correlate2d

sys.path.append(str(Path.cwd().parent))

plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (14, 8)

np.random.seed(42)

print("✅ Ready for CNNs!")

## 1️⃣ Problème avec les MLP pour les Images

### 🤔 Rappel : Notre réseau MLP

```
Image 28×28 → Aplatir en 784 → [784 → 128 → 10]
```

### ❌ Problèmes :

1. **Perte de structure spatiale**
   - L'image est 2D, mais on la traite comme 1D
   - Les pixels voisins sont importants (contours, formes)
   - On casse cette relation !

2. **Trop de paramètres**
   - 784 × 128 = 100,352 poids rien que pour la première couche !
   - Pour une image 224×224 : 50,176 × 128 = 6.4 millions 😱

3. **Pas invariant à la translation**
   - Si le chiffre est décalé de 2 pixels → nouveau pattern à apprendre
   - Pas efficace !

### ✅ Solution : CNN !

Les CNN résolvent tous ces problèmes !

## 2️⃣ L'Opération de Convolution

### 💡 L'Idée

Au lieu de connecter chaque pixel à tous les neurones, on utilise un **petit filtre** qui glisse sur l'image.

### 🔍 Exemple Simple

**Image 5×5** et **Filtre 3×3** :

```
Image:              Filtre (détecteur de bord vertical):
┌─────────────┐     ┌─────────┐
│ 1 1 1 0 0 │     │ 1  0 -1 │
│ 1 1 1 0 0 │     │ 1  0 -1 │
│ 1 1 1 0 0 │     │ 1  0 -1 │
│ 1 1 1 0 0 │     └─────────┘
│ 1 1 1 0 0 │
└─────────────┘

Résultat de convolution → Détecte le bord vertical !
```

### 📐 Formule Mathématique

$$
\text{Output}(i, j) = \sum_{m}\sum_{n} \text{Image}(i+m, j+n) \cdot \text{Filtre}(m, n)
$$

In [ ]:
def convolution_2d(image, kernel):
    """
    Applique une convolution 2D
    
    Args:
        image: image d'entrée (H, W)
        kernel: filtre de convolution (kH, kW)
    
    Returns:
        output: résultat de la convolution
    """
    # Mode 'valid' = pas de padding
    return correlate2d(image, kernel, mode='valid')

# Créer une image simple avec un bord vertical
image = np.array([
    [0, 0, 0, 1, 1, 1, 1],
    [0, 0, 0, 1, 1, 1, 1],
    [0, 0, 0, 1, 1, 1, 1],
    [0, 0, 0, 1, 1, 1, 1],
    [0, 0, 0, 1, 1, 1, 1],
    [0, 0, 0, 1, 1, 1, 1],
    [0, 0, 0, 1, 1, 1, 1],
])

# Filtre de détection de bord vertical
vertical_edge = np.array([
    [1,  0, -1],
    [1,  0, -1],
    [1,  0, -1]
])

# Filtre de détection de bord horizontal
horizontal_edge = np.array([
    [ 1,  1,  1],
    [ 0,  0,  0],
    [-1, -1, -1]
])

# Appliquer les convolutions
output_vertical = convolution_2d(image, vertical_edge)
output_horizontal = convolution_2d(image, horizontal_edge)

# Visualiser
fig, axes = plt.subplots(2, 3, figsize=(16, 10))
fig.suptitle('🔍 Opération de Convolution', fontsize=18, fontweight='bold')

# Ligne 1 : Bord vertical
axes[0, 0].imshow(image, cmap='gray', vmin=0, vmax=1)
axes[0, 0].set_title('Image Originale', fontsize=14, fontweight='bold')
axes[0, 0].axis('off')

axes[0, 1].imshow(vertical_edge, cmap='RdBu', vmin=-1, vmax=1)
axes[0, 1].set_title('Filtre: Bord Vertical', fontsize=14, fontweight='bold')
axes[0, 1].axis('off')
# Ajouter les valeurs
for i in range(3):
    for j in range(3):
        axes[0, 1].text(j, i, f'{vertical_edge[i,j]}', 
                       ha='center', va='center', color='white', fontsize=14, fontweight='bold')

im1 = axes[0, 2].imshow(output_vertical, cmap='hot')
axes[0, 2].set_title('Résultat: Bord Détecté! ✅', fontsize=14, fontweight='bold', color='green')
axes[0, 2].axis('off')
plt.colorbar(im1, ax=axes[0, 2], fraction=0.046)

# Ligne 2 : Bord horizontal
axes[1, 0].imshow(image, cmap='gray', vmin=0, vmax=1)
axes[1, 0].set_title('Image Originale', fontsize=14, fontweight='bold')
axes[1, 0].axis('off')

axes[1, 1].imshow(horizontal_edge, cmap='RdBu', vmin=-1, vmax=1)
axes[1, 1].set_title('Filtre: Bord Horizontal', fontsize=14, fontweight='bold')
axes[1, 1].axis('off')
for i in range(3):
    for j in range(3):
        axes[1, 1].text(j, i, f'{horizontal_edge[i,j]}', 
                       ha='center', va='center', color='white', fontsize=14, fontweight='bold')

im2 = axes[1, 2].imshow(output_horizontal, cmap='hot')
axes[1, 2].set_title('Résultat: Pas de bord ❌', fontsize=14, fontweight='bold', color='red')
axes[1, 2].axis('off')
plt.colorbar(im2, ax=axes[1, 2], fraction=0.046)

plt.tight_layout()
plt.show()

print("\n💡 Observation:")
print("   • Le filtre vertical détecte le bord vertical (activation forte)")
print("   • Le filtre horizontal ne détecte rien (activation faible)")
print("   • Les CNN apprennent automatiquement ces filtres ! 🎯")

### 🧪 Test sur une Vraie Image MNIST

In [ ]:
from src.utils import load_mnist_data

# Charger MNIST
X_train, y_train, X_test, y_test = load_mnist_data()

# Prendre une image
image_mnist = X_train[0].reshape(28, 28)
label = y_train[0]

# Différents filtres
filters = {
    'Bord Vertical': np.array([[1, 0, -1], [2, 0, -2], [1, 0, -1]]),  # Sobel vertical
    'Bord Horizontal': np.array([[1, 2, 1], [0, 0, 0], [-1, -2, -1]]),  # Sobel horizontal
    'Diagonal': np.array([[2, 1, 0], [1, 0, -1], [0, -1, -2]]),
    'Flou': np.ones((3, 3)) / 9,  # Moyenne
    'Accentuation': np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]]),  # Sharpen
}

fig, axes = plt.subplots(2, 3, figsize=(16, 10))
fig.suptitle(f'🔍 Filtres de Convolution sur le Chiffre "{label}"', fontsize=18, fontweight='bold')

# Image originale
axes[0, 0].imshow(image_mnist, cmap='gray_r')
axes[0, 0].set_title('Image Originale', fontsize=14, fontweight='bold')
axes[0, 0].axis('off')

# Appliquer les filtres
for idx, (name, kernel) in enumerate(filters.items(), 1):
    row = idx // 3
    col = idx % 3
    
    filtered = convolution_2d(image_mnist, kernel)
    
    axes[row, col].imshow(filtered, cmap='viridis')
    axes[row, col].set_title(name, fontsize=13, fontweight='bold')
    axes[row, col].axis('off')

plt.tight_layout()
plt.show()

print("\n🎨 Différents filtres détectent différentes features:")
print("   • Bords verticaux/horizontaux: contours du chiffre")
print("   • Diagonale: traits obliques")
print("   • Flou: réduit le bruit")
print("   • Accentuation: renforce les détails")
print("\n   ➡️ Un CNN apprend automatiquement les meilleurs filtres ! 🧠")

## 3️⃣ Pooling (Sous-échantillonnage)

### 💡 L'Idée

Réduire la taille de l'image pour :
- Moins de calculs
- Invariance à de petites translations
- Réduire l'overfitting

### 📊 Max Pooling 2×2

```
Input (4×4):           Output (2×2):
┌───────────────┐      ┌───────┐
│ 1  3 │ 2  4 │      │ 3 │ 4 │
│ 5  6 │ 7  8 │  →   ├───┼───┤
├──────┼──────┤      │ 9 │ 12│
│ 2  1 │ 3  2 │      └───────┘
│ 4  9 │ 11 12│
└───────────────┘

Prend le max de chaque région 2×2
```

In [ ]:
def max_pooling_2d(image, pool_size=2):
    """
    Max pooling 2D
    """
    h, w = image.shape
    new_h, new_w = h // pool_size, w // pool_size
    
    output = np.zeros((new_h, new_w))
    
    for i in range(new_h):
        for j in range(new_w):
            # Prendre le max dans la fenêtre pool_size × pool_size
            window = image[i*pool_size:(i+1)*pool_size, j*pool_size:(j+1)*pool_size]
            output[i, j] = np.max(window)
    
    return output

# Test sur l'image MNIST
image_original = X_train[0].reshape(28, 28)
image_pooled = max_pooling_2d(image_original, pool_size=2)

fig, axes = plt.subplots(1, 3, figsize=(16, 5))
fig.suptitle('🔽 Max Pooling', fontsize=18, fontweight='bold')

axes[0].imshow(image_original, cmap='gray_r')
axes[0].set_title(f'Original: 28×28 = {28*28} pixels', fontsize=14, fontweight='bold')
axes[0].axis('off')

axes[1].imshow(image_pooled, cmap='gray_r')
axes[1].set_title(f'Pooling 2×2: 14×14 = {14*14} pixels', fontsize=14, fontweight='bold')
axes[1].axis('off')

# Pooling 4×4
image_pooled_4 = max_pooling_2d(image_original, pool_size=4)
axes[2].imshow(image_pooled_4, cmap='gray_r')
axes[2].set_title(f'Pooling 4×4: 7×7 = {7*7} pixels', fontsize=14, fontweight='bold')
axes[2].axis('off')

plt.tight_layout()
plt.show()

print("\n📉 Réduction de dimensionnalité:")
print(f"   • Original: {28*28} pixels (100%)")
print(f"   • Pooling 2×2: {14*14} pixels ({14*14/(28*28)*100:.0f}%)")
print(f"   • Pooling 4×4: {7*7} pixels ({7*7/(28*28)*100:.0f}%)")
print("\n   💡 Le chiffre reste reconnaissable !")

## 4️⃣ Architecture CNN Typique

### 🏗️ Structure

```
Input (28×28×1)
    ↓
Conv1: 8 filtres 3×3 + ReLU → (26×26×8)
    ↓
MaxPool 2×2 → (13×13×8)
    ↓
Conv2: 16 filtres 3×3 + ReLU → (11×11×16)
    ↓
MaxPool 2×2 → (5×5×16)
    ↓
Flatten → (400)
    ↓
Dense(64) + ReLU
    ↓
Dense(10) + Softmax → Prédiction
```

### 📊 Avantages

- **Moins de paramètres** : ~20k vs 100k pour un MLP
- **Meilleure performance** : 98-99% vs 95-96%
- **Invariance locale** : détecte les features partout dans l'image

## 5️⃣ Implémentation Simplifiée d'un CNN

Créons une version simple pour comprendre !

In [ ]:
class SimpleCNN:
    """
    CNN simple pour comprendre les concepts
    
    Note: Utilise scipy pour les convolutions (plus simple)
    En pratique, on utiliserait PyTorch/TensorFlow
    """
    
    def __init__(self, num_filters=8, filter_size=3):
        self.num_filters = num_filters
        self.filter_size = filter_size
        
        # Initialiser les filtres aléatoirement (petit)
        self.filters = np.random.randn(num_filters, filter_size, filter_size) * 0.1
        
        print(f"✅ CNN créé avec {num_filters} filtres {filter_size}×{filter_size}")
    
    def forward_conv(self, image):
        """
        Applique la convolution
        """
        h, w = image.shape
        output_h = h - self.filter_size + 1
        output_w = w - self.filter_size + 1
        
        # Créer le volume de sortie
        output = np.zeros((output_h, output_w, self.num_filters))
        
        # Appliquer chaque filtre
        for f in range(self.num_filters):
            output[:, :, f] = correlate2d(image, self.filters[f], mode='valid')
        
        return output
    
    def relu(self, x):
        return np.maximum(0, x)
    
    def forward(self, image, visualize=False):
        """
        Forward pass complet
        """
        # Conv + ReLU
        conv_output = self.forward_conv(image)
        activated = self.relu(conv_output)
        
        # Max pooling
        pooled_outputs = []
        for f in range(self.num_filters):
            pooled = max_pooling_2d(activated[:, :, f], pool_size=2)
            pooled_outputs.append(pooled)
        
        pooled = np.stack(pooled_outputs, axis=-1)
        
        if visualize:
            self.visualize_features(image, conv_output, activated, pooled)
        
        return pooled
    
    def visualize_features(self, original, conv_output, activated, pooled):
        """
        Visualise les feature maps
        """
        n_filters = min(8, self.num_filters)  # Max 8 pour l'affichage
        
        fig, axes = plt.subplots(4, n_filters + 1, figsize=(18, 10))
        fig.suptitle('🎨 Visualisation des Feature Maps', fontsize=18, fontweight='bold')
        
        # Ligne 1: Filtres
        axes[0, 0].text(0.5, 0.5, 'Filtres\nAppris', ha='center', va='center', 
                       fontsize=14, fontweight='bold', transform=axes[0, 0].transAxes)
        axes[0, 0].axis('off')
        for i in range(n_filters):
            axes[0, i+1].imshow(self.filters[i], cmap='RdBu', vmin=-0.5, vmax=0.5)
            axes[0, i+1].set_title(f'Filtre {i+1}', fontsize=10)
            axes[0, i+1].axis('off')
        
        # Ligne 2: Convolution
        axes[1, 0].imshow(original, cmap='gray_r')
        axes[1, 0].set_title('Original', fontsize=11, fontweight='bold')
        axes[1, 0].axis('off')
        for i in range(n_filters):
            axes[1, i+1].imshow(conv_output[:, :, i], cmap='viridis')
            axes[1, i+1].set_title(f'Conv {i+1}', fontsize=10)
            axes[1, i+1].axis('off')
        
        # Ligne 3: Après ReLU
        axes[2, 0].text(0.5, 0.5, 'Après\nReLU', ha='center', va='center',
                       fontsize=14, fontweight='bold', transform=axes[2, 0].transAxes)
        axes[2, 0].axis('off')
        for i in range(n_filters):
            axes[2, i+1].imshow(activated[:, :, i], cmap='viridis')
            axes[2, i+1].set_title(f'ReLU {i+1}', fontsize=10)
            axes[2, i+1].axis('off')
        
        # Ligne 4: Après Pooling
        axes[3, 0].text(0.5, 0.5, 'Après\nPooling', ha='center', va='center',
                       fontsize=14, fontweight='bold', transform=axes[3, 0].transAxes)
        axes[3, 0].axis('off')
        for i in range(n_filters):
            axes[3, i+1].imshow(pooled[:, :, i], cmap='viridis')
            axes[3, i+1].set_title(f'Pool {i+1}', fontsize=10)
            axes[3, i+1].axis('off')
        
        plt.tight_layout()
        plt.show()

# Créer et tester
cnn = SimpleCNN(num_filters=8, filter_size=3)

# Test sur une image
test_image = X_train[0].reshape(28, 28)
output = cnn.forward(test_image, visualize=True)

print(f"\n📊 Dimensions:")
print(f"   Input: {test_image.shape}")
print(f"   Output: {output.shape}")
print(f"\n💡 Chaque filtre a appris à détecter une feature différente !")

## 6️⃣ Comparaison MLP vs CNN

### 📊 Tableau Comparatif

| Critère | MLP (784→128→10) | CNN Simple | CNN Optimisé |
|---------|------------------|------------|---------------|
| **Paramètres** | ~100,000 | ~20,000 | ~50,000 |
| **Accuracy (MNIST)** | 95-96% | 97-98% | 98-99%+ |
| **Temps d'entraînement** | Rapide | Moyen | Lent |
| **Invariance spatiale** | ❌ Non | ✅ Oui | ✅ Oui |
| **Scalabilité images** | ❌ Mauvaise | ✅ Bonne | ✅ Excellente |

### 🏆 Verdict

- **MLP** : Bon pour apprendre les concepts, mais limité
- **CNN** : Standard pour la vision par ordinateur
- **État de l'art** : CNN profonds (ResNet, EfficientNet, etc.)

## 🎯 Récapitulatif

### ✅ Ce que tu as appris

1. **Limites des MLP** pour les images
   - Perte de structure spatiale
   - Trop de paramètres
   - Pas d'invariance

2. **Convolution**
   - Filtres qui glissent sur l'image
   - Détection de features locales
   - Apprentissage automatique des filtres

3. **Pooling**
   - Réduction de dimensionnalité
   - Invariance aux petites translations
   - Moins de calculs

4. **Architecture CNN**
   - Conv → ReLU → Pool → Conv → ReLU → Pool → Dense
   - Meilleure performance avec moins de paramètres

### 💡 Points clés

- 🖼️ **CNN** = spécialisé pour les images
- 🔍 **Convolution** = détection de patterns locaux
- 📉 **Pooling** = réduction et invariance
- 🏆 **Performance** = 98-99% sur MNIST !

### 🚀 Prochaines Étapes

Pour aller plus loin :
- Implémenter un vrai CNN avec PyTorch/TensorFlow
- Essayer sur des images couleur (CIFAR-10)
- Architectures modernes (ResNet, VGG, etc.)
- Transfer learning avec des modèles pré-entraînés

---

**Tu comprends maintenant pourquoi les CNN dominent la vision par ordinateur ! 🎉**